In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [75]:
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import sklearn
%matplotlib inline
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from scipy import stats
import sklearn.metrics as sm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor

Import Dataset:

In [76]:
item_set = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
item_category_set = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
train_set = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
shop_set = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test_set = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [77]:
# item set
item_set.head()

In [78]:
item_set.shape

In [79]:
item_set["item_name"].nunique()

In [80]:
#item category set
item_category_set.head()

In [81]:
item_category_set.shape

In [82]:
item_category_set['item_category_name'].nunique()

In [83]:
#shop set
shop_set.head()

In [84]:
shop_set.shape

In [85]:
shop_set['shop_name'].nunique()

In [86]:
#train set
train_set.head(20)

In [87]:
item_price = train_set['item_price']
item_price.sort_values( ascending=False)

In [88]:
train_set.shape

In [89]:
train_set.info()

In [90]:
train_set.describe()

Check null values:

In [91]:
train_set.isnull().sum()

Data Visualization:

In [92]:
train_set.hist(figsize=(15,15), bins=6)
plt.show()

Item Price:

In [93]:
train_set['item_cnt_day'].hist(range=[-1, 10], facecolor='green', align='mid')
plt.show()

In [94]:
sns.displot(train_set['item_cnt_day'])

In [95]:
train_set['item_cnt_day'].describe()

In [96]:
correlation_matrix = train_set.corr()
correlation_matrix['item_cnt_day'].sort_values(ascending = False)

In [97]:
correlation_num = 6
correlation_cols = correlation_matrix.nlargest(correlation_num,'item_cnt_day')['item_cnt_day'].index
correlation_mat_sales = np.corrcoef(train_set[correlation_cols].values.T)
sns.set(font_scale=1.25)
f, ax = plt.subplots(figsize=(12, 9))
hm = sns.heatmap(correlation_mat_sales, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 7}, yticklabels=correlation_cols.values, xticklabels=correlation_cols.values)
plt.show()

In [98]:
train_set = train_set.drop(columns=['date','date_block_num','item_price'])

In [99]:
train_set.head()

In [100]:
y= train_set['item_cnt_day']
x= train_set.drop(columns=['item_cnt_day'])
print(len(x.columns))

Split Dataset:

In [101]:
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size = 0.3, random_state = 60,shuffle=True)
print(len(X_train))
print(len(X_test))

**Train model with train data**

Linear Regression:

In [102]:
linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)

#make predictions using linear regression
price_predict =(linear_model.predict(X_test))

#Measure Performance 
linear_mse = mean_squared_error(Y_test,price_predict)
print("Mean Squared error: ",linear_mse)
linear_SqMse = np.sqrt(linear_mse)
print("Root Mean Squared error: ", linear_SqMse)

print("Explain variance score for Linear Regression =", round(sm.explained_variance_score(Y_test,price_predict),2)) 
print("R2 score for Linear Regression =", round(sm.r2_score(Y_test, price_predict), 2))

In [103]:
lasso_model= Lasso(max_iter=5000)

# Train the model with training data
lasso_model.fit(X_train,Y_train)
#make predictions 
price_predict =(lasso_model.predict(X_test))


lasso_mse = mean_squared_error(Y_test,price_predict)
print("Mean Squared Error : " , lasso_mse)
lasso_SqMse = np.sqrt(lasso_mse)
print("Root Mean Squared error: ", lasso_SqMse)

print("Explain variance score for Lasso Regression =", round(sm.explained_variance_score(Y_test,price_predict), 2)) 
print("R2 score for Lasso Regression =", round(sm.r2_score(Y_test, price_predict), 2))

In [104]:
# Create a model with 50 decision trees
forest_model= RandomForestRegressor(n_estimators = 50, random_state = 42)

# Train the model with training data
forest_model.fit(X_train,Y_train)
#make predictions using random forest model
price_predict =(forest_model.predict(X_test))


forest_mse = mean_squared_error(Y_test,price_predict)
print("Mean Squared Error : " , forest_mse)
forest_SqMse = np.sqrt(forest_mse)
print("Root Mean Squared error: ", forest_SqMse)

print("Explain variance score for Random Forest Regression =", round(sm.explained_variance_score(Y_test,price_predict), 2)) 
print("R2 score for Random Forest Regression =", round(sm.r2_score(Y_test, price_predict), 2))

In [105]:
# Create a model with 50 decision trees
xgbr=  XGBRegressor()

# Train the model with training data
xgbr.fit(X_train,Y_train)
#make predictions using random forest model
price_predict =(xgbr.predict(X_test))


xgbr_mse = mean_squared_error(Y_test,price_predict)
print("Mean Squared Error : " , xgbr_mse)
xgbr_SqMse = np.sqrt(xgbr_mse)
print("Root Mean Squared error: ", xgbr_SqMse)

print("Explain variance score for XGB Regression =", round(sm.explained_variance_score(Y_test,price_predict), 2)) 
print("R2 score for XGB Regression =", round(sm.r2_score(Y_test, price_predict), 2))

**Test set**

In [106]:
test_set.head()

In [107]:
test_id = test_set['ID']
test_x= test_set.drop(columns=['ID'])
print(len(test_x.columns))

In [108]:
y_prdict = forest_model.predict(test_x)

In [109]:
submission = pd.DataFrame(list(zip(test_id, y_prdict)),
               columns =['ID', 'item_cnt_month'])
submission.head(20)

In [110]:
submission.to_csv('submission.csv', index=False)